# Setup

In [ ]:
# import packages
import pandas as pd
import pathlib
from pathlib import Path
import os
import arcpy
from utils import *
import numpy as np
import pickle
# external connection packages
from sqlalchemy.engine import URL
from sqlalchemy import create_engine

# pandas options
pd.options.mode.copy_on_write = True
pd.options.mode.chained_assignment = None
pd.options.display.max_columns = 999
pd.options.display.max_rows    = 999

# my workspace 
workspace = r"C:\Users\mbindl\Desktop\Workspace.gdb"
# current working directory
local_path = pathlib.Path().absolute()
# set data path as a subfolder of the current working directory TravelDemandModel\2022\
data_dir = local_path.parents[0] / 'data'
# folder to save processed data
out_dir  = local_path.parents[0] / 'data/processed_data'
# workspace gdb for stuff that doesnt work in memory
# gdb = os.path.join(local_path,'Workspace.gdb')
gdb = workspace
# set environement workspace to in memory 
arcpy.env.workspace = 'memory'
# # clear memory workspace
# arcpy.management.Delete('memory')

# overwrite true
arcpy.env.overwriteOutput = True
# Set spatial reference to NAD 1983 UTM Zone 10N
sr = arcpy.SpatialReference(26910)

# get parcels from the database
# network path to connection files
filePath = "F:/GIS/PARCELUPDATE/Workspace/"
# database file path 
sdeBase    = os.path.join(filePath, "Vector.sde")
sdeCollect = os.path.join(filePath, "Collection.sde")
sdeTabular = os.path.join(filePath, "Tabular.sde")
sdeEdit    = os.path.join(filePath, "Edit.sde")

# Pickle variables
# part 1 - spatial joins and new categorical fields
parcel_pickle_part1    = data_dir / 'parcel_pickle1.pkl'
# part 2 - forecasting applied
parcel_pickle_part2    = data_dir / 'parcel_pickle2.pkl'


In [ ]:
##  Fields to Keep

# columsn to list
initial_columns = [ 'APN',
                    'APO_ADDRESS',
                    'Residential_Units',
                    'TouristAccommodation_Units',
                    'CommercialFloorArea_SqFt',
                    'YEAR',
                    'JURISDICTION',
                    'COUNTY',
                    'OWNERSHIP_TYPE',
                    'COUNTY_LANDUSE_DESCRIPTION',
                    'EXISTING_LANDUSE',
                    'REGIONAL_LANDUSE',
                    'YEAR_BUILT',
                    'PLAN_ID',
                    'PLAN_NAME',
                    'ZONING_ID',
                    'ZONING_DESCRIPTION',
                    'TOWN_CENTER',
                    'LOCATION_TO_TOWNCENTER',
                    'TAZ',
                    'PARCEL_ACRES',
                    'PARCEL_SQFT',
                    'WITHIN_BONUSUNIT_BNDY',
                    'WITHIN_TRPA_BNDY',
                    'SHAPE']

# parcel master fields
pm_fields = ['AS_LANDVALUE',
                'AS_IMPROVALUE',
                'AS_SUM',
                'TAX_LANDVALUE',
                'TAX_IMPROVALUE',
                'TAX_SUM',
                'TAX_YEAR',
                'YEAR_BUILT',
                'UNITS',
                'BEDROOMS',
                'BATHROOMS',
                'BUILDING_SQFT',
                'ESTIMATED_COVERAGE_ALLOWED',
                'IMPERVIOUS_SURFACE_SQFT']


# schema for the final output
final_schema = ['APN', 
                
                'Residential_Units', 
                'TouristAccommodation_Units', 
                'CommercialFloorArea_SqFt',
                
                'RoomsRented_PerDay', 
                'VHR_Occupancy_Rate',
                'TAU_Occupancy_Rate', 
                'PrimaryResidence_Rate', 
                'SecondaryResidence_Rate',
                'HighIncome_Rate',	
                'MediumIncome_Rate', 
                'LowIncome_Rate', 
                'PersonsPerUnit',

                'APO_ADDRESS',
                'YEAR_BUILT',
                'JURISDICTION', 
                'COUNTY', 
                'OWNERSHIP_TYPE',
                'COUNTY_LANDUSE_DESCRIPTION',
                'EXISTING_LANDUSE',
                'REGIONAL_LANDUSE',
                'VHR', 
                'BLOCK_GROUP', 
                'TAZ', 
                'OCCUPANCY_ZONE', 
                'PLAN_ID',
                'PLAN_NAME',
                'ZONING_ID',
                'ZONING_DESCRIPTION',
                'TOWN_CENTER',
                'LOCATION_TO_TOWNCENTER',
                'TAZ',
                'WITHIN_BONUSUNIT_BNDY',
                'WITHIN_TRPA_BNDY',
                'IPES_SCORE',
                'IPES_SCORE_TYPE',
                'RETIRED',
                'HOUSING_ZONING',
                'COMMERCIAL_ALLOWED',
                'TOURIST_ALLOWED',
                'ADU_ALLOWED',
                'DENSITY',
                'MAX_RESIDENTIAL_UNITS',
                'MAX_UNITS',
                'POTENTIAL_UNITS',
                'TOP_TEN_POTENTIAL_UNITS',
                'FORECASTED_RESIDENTIAL_UNITS',
                'FORECASTED_COMMERCIAL_SQFT',
                'FORECASTED_TOURIST_UNITS',
                'FORECAST_REASON',
                'FORECASTED_RES_OCCUPANCY_RATE',    
                'PARCEL_ACRES', 'PARCEL_SQFT', 'SHAPE']


#### Functions

In [ ]:
# sql server connection
def get_conn(db):
    # Get database user and password from environment variables on machine running script
    db_user             = os.environ.get('DB_USER')
    db_password         = os.environ.get('DB_PASSWORD')
    # driver is the ODBC driver for SQL Server
    driver              = 'ODBC Driver 17 for SQL Server'
    # server names are
    sql_12              = 'sql12'
    sql_14              = 'sql14'
    # make it case insensitive
    db = db.lower()
    # make sql database connection with pyodbc
    if db   == 'sde_tabular':
        connection_string = f"DRIVER={driver};SERVER={sql_12};DATABASE={db};UID={db_user};PWD={db_password}"
        connection_url = URL.create("mssql+pyodbc", query={"odbc_connect": connection_string})
        engine = create_engine(connection_url)
    elif db == 'tahoebmpsde':
        connection_string = f"DRIVER={driver};SERVER={sql_14};DATABASE={db};UID={db_user};PWD={db_password}"
        connection_url = URL.create("mssql+pyodbc", query={"odbc_connect": connection_string})
        engine = create_engine(connection_url)
    elif db == 'sde':
        connection_string = f"DRIVER={driver};SERVER={sql_12};DATABASE={db};UID={db_user};PWD={db_password}"
        connection_url = URL.create("mssql+pyodbc", query={"odbc_connect": connection_string})
        engine = create_engine(connection_url)
    # else return None
    else:
        engine = None
    # connection file to use in pd.read_sql
    return engine

# save to pickle
def to_pickle(data, filename):
    with open(filename, 'wb') as f:
        pickle.dump(data, f)
    print(f'{filename} pickled')

# save to pickle and feature class
def to_pickle_fc(data, filename):
    data.spatial.to_featureclass(filename)
    with open(filename, 'wb') as f:
        pickle.dump(data, f)
    print(f'{filename} pickled and saved as feature class')

# get a pickled file as a dataframe
def from_pickle(filename):
    with open(filename, 'rb') as f:
        data = pickle.load(f)
    print(f'{filename} unpickled')
    return data
def get_commercial_zones(df):
    columns_to_keep = ['Zoning_ID', 'Category', 'Density']
    # filter Use_Type to Multiple Family Dwelling
    df = df.loc[df['Category'] == 'Commercial']
    return df[columns_to_keep]

def get_tourist_zones(df):
    columns_to_keep = ['Zoning_ID', 'Category', 'Density']
    # filter Use_Type to Multiple Family Dwelling
    df = df.loc[df['Category'] == 'Tourist Accommodation']
    return df[columns_to_keep]

# function to get where Zoningin_ID Use_Type = Multi-Family and Density
def get_mf_zones(df):
    columns_to_keep = ['Zoning_ID', 'Use_Type', 'Density']
    # filter Use_Type to Multiple Family Dwelling
    df = df.loc[df['Use_Type'] == 'Multiple Family Dwelling']
    return df[columns_to_keep]

# function to get where Zoningin_ID Use_Type = Multi-Family and Density
def get_sf_zones(df):
    columns_to_keep = ['Zoning_ID', 'Use_Type', 'Density']
    # filter Use_Type to Multiple Family Dwelling
    df = df.loc[df['Use_Type'] == 'Single Family Dwelling']
    return df[columns_to_keep]

def get_mf_only_zones(df):
    columns_to_keep = ['Zoning_ID', 'Use_Type', 'Density']
    # filter Use_Type to Single Family Dwelling and not Multiple Family Dwelling
    dfMF = get_mf_zones(df)
    dfSF = get_sf_zones(df)
    # get Zoning_ID that are in both dataframes
    df = dfMF.loc[~dfMF['Zoning_ID'].isin(dfSF['Zoning_ID'])]
    return df[columns_to_keep]

def get_sf_only_zones(df):
    columns_to_keep = ['Zoning_ID', 'Use_Type', 'Density']
    # filter Use_Type to Single Family Dwelling and not Multiple Family Dwelling
    dfMF = get_mf_zones(df)
    dfSF = get_sf_zones(df)
    df = dfSF.loc[~dfSF['Zoning_ID'].isin(dfMF['Zoning_ID'])]
    return df[columns_to_keep]

def get_sf_mf_zones(df):
    columns_to_keep = ['Zoning_ID', 'Use_Type', 'Density']
    # get SF and MF zones
    dfSF = get_sf_zones(df)
    dfMF = get_mf_zones(df)
    # add the two dataframes together
    df = pd.concat([dfSF, dfMF])
    # only keep duplicate Zoning_ID
    df = df[df.duplicated(subset=['Zoning_ID'], keep=False)]
    return df[columns_to_keep]

def get_recieving_zones(df):
    columns_to_keep = ['Zoning_ID', 'SPECIAL_DESIGNATION']
    # filter transfer recieving
    df = df.loc[df['SPECIAL_DESIGNATION'] == 'Receive']
    return df[columns_to_keep]

def get_sending_zones(df):
    columns_to_keep = ['Zoning_ID', 'SPECIAL_DESIGNATION']
    df = df.loc[df['SPECIAL_DESIGNATION'] == 'Transfer']
    return df[columns_to_keep]


### Get Data

In [ ]:
# get parcel master
parcelmaster_db = Path(sdeBase) / "SDE.Parcels\SDE.Parcel_Master"
sdf_master = pd.DataFrame.spatial.from_featureclass(parcelmaster_db)
sdf_master.spatial.sr = sr

In [ ]:
sdf_master.columns.to_list()

In [ ]:
# parcel development layer polygons
parcel_db = Path(sdeEdit) / "SDE.Parcel\\SDE.Parcel_History_Attributed"
# query 2022 rows
sdf_units = pd.DataFrame.spatial.from_featureclass(parcel_db)
sdf_units = sdf_units.loc[sdf_units['YEAR'] == 2022]
sdf_units.spatial.sr = sr

# get parcel master
parcelmaster_db = Path(sdeBase) / "SDE.Parcel\\SDE.Parcel_Master"
sdf_master = pd.DataFrame.spatial.from_featureclass(parcelmaster_db)
sdf_master.spatial.sr = sr

# # get parcel level data from Collection SDE
# vhr feature layer polygons 
vhr_db = Path(sdeCollect) / "SDE.Parcel\\SDE.Parcel_VHR"
sdf_vhr = pd.DataFrame.spatial.from_featureclass(vhr_db)
sdf_vhr.spatial.sr = sr
# filter vhr layer to active status
sdf_vhr = sdf_vhr.loc[sdf_vhr['Status'] == 'Active']

# TAZ feature layer polygons
taz_db = Path(sdeBase) / "SDE.Transportation\\SDE.Transportation_Analysis_Zone"
# get as spatial dataframe
sdf_taz = pd.DataFrame.spatial.from_featureclass(taz_db)
# set spatial reference to NAD 1983 UTM Zone 10N
sdf_taz.spatial.sr = sr

# censuse feature class
census_fc    = Path(sdeBase) / "SDE.Census\\SDE.Tahoe_Census_Geography"
# bouns unit boundary feature class
bonus_unit_fc = Path(sdeBase) / "SDE.Planning\SDE.Bonus_unit_boundary"

# disable Z values on block group feature layer
with arcpy.EnvManager(outputZFlag="Disabled"):    
    arcpy.conversion.FeatureClassToGeodatabase(
        Input_Features="F:\GIS\DB_CONNECT\Vector.sde\SDE.Census\SDE.Tahoe_Census_Geography",
        Output_Geodatabase=r"C:\Users\mbindl\Desktop\Workspace.gdb"
    )
# disable Z values on block group feature layer
with arcpy.EnvManager(outputZFlag="Disabled"):    
    arcpy.conversion.FeatureClassToGeodatabase(
        Input_Features="F:\GIS\DB_CONNECT\Vector.sde\SDE.Planning\SDE.Bonus_unit_boundary",
        Output_Geodatabase=r"C:\Users\mbindl\Desktop\Workspace.gdb"
    )

# block group feature layer polygons with no Z
sdf_block = pd.DataFrame.spatial.from_featureclass(Path(gdb) / 'Tahoe_Census_Geography')
sdf_block = sdf_block.loc[(sdf_block['YEAR'] == 2020) & (sdf_block['GEOGRAPHY'] == 'Block Group')]
sdf_block.spatial.sr = sr

# bonus unit boundary wihtout Z
sdf_bonus = pd.DataFrame.spatial.from_featureclass(Path(gdb) / 'Bonus_unit_boundary')
sdf_bonus.spatial.sr = sr

# get parcel level data from LTinfo
dfIPES       = pd.read_json("https://www.laketahoeinfo.org/WebServices/GetParcelIPESScores/JSON/e17aeb86-85e3-4260-83fd-a2b32501c476")
dfLCV_LTinfo = pd.read_json('https://www.laketahoeinfo.org/WebServices/GetParcelsByLandCapability/JSON/e17aeb86-85e3-4260-83fd-a2b32501c476')
dfRetired    = pd.read_json("https://www.laketahoeinfo.org/WebServices/GetAllParcels/JSON/e17aeb86-85e3-4260-83fd-a2b32501c476")
dfBankedDev  = pd.read_json('https://www.laketahoeinfo.org/WebServices/GetBankedDevelopmentRights/JSON/e17aeb86-85e3-4260-83fd-a2b32501c476')
dfTransacted = pd.read_json('https://www.laketahoeinfo.org/WebServices/GetTransactedAndBankedDevelopmentRights/JSON/e17aeb86-85e3-4260-83fd-a2b32501c476')
dfAllParcels = pd.read_json('https://www.laketahoeinfo.org/WebServices/GetAllParcels/JSON/e17aeb86-85e3-4260-83fd-a2b32501c476')

# get use tables 
# zoning data
sde_engine = get_conn('sde')
with sde_engine.begin() as conn:
    df_uses    = pd.read_sql("SELECT * FROM sde.SDE.PermissibleUses", conn)
    df_special = pd.read_sql("SELECT * FROM sde.SDE.Special_Designation", conn)

### Parcel Data Engineering

In [ ]:
# spatial join to get TAZ
arcpy.SpatialJoin_analysis(sdf_units, sdf_taz, "Existing_Development_TAZ", 
                           "JOIN_ONE_TO_ONE", "KEEP_ALL", "", "HAVE_THEIR_CENTER_IN")
# spatial join to get Block Group
arcpy.SpatialJoin_analysis(sdf_units, sdf_block, "Existing_Development_BlockGroup", 
                           "JOIN_ONE_TO_ONE", "KEEP_ALL", "", "HAVE_THEIR_CENTER_IN")
# spatial join of Bonus Unit Boundary
arcpy.SpatialJoin_analysis(sdf_units, sdf_bonus, "Existing_Development_BonusUnitBoundary",
                            "JOIN_ONE_TO_ONE", "KEEP_ALL", "", "INTERSECT")

In [ ]:
# spatial dataframe with only initial columns
sdfParcels = sdf_units[initial_columns]

# get results of spatial joins as spatial dataframes
sdf_units_taz   = pd.DataFrame.spatial.from_featureclass("Existing_Development_TAZ", sr=sr)  
sdf_units_block = pd.DataFrame.spatial.from_featureclass("Existing_Development_BlockGroup", sr=sr)
sdf_units_bonus = pd.DataFrame.spatial.from_featureclass("Existing_Development_BonusUnitBoundary", sr=sr)
# cast to string
sdf_units_bonus['WITHIN_BONUSUNIT_BNDY'] = sdf_units_bonus['WITHIN_BONUSUNIT_BNDY'].astype(str)
sdf_units_bonus['WITHIN_BONUSUNIT_BNDY'] = 'No'
# if Id is not NA then within bonus unit boundary = yes, else
sdf_units_bonus.loc[sdf_units_bonus['Id'].notna(), 'WITHIN_BONUSUNIT_BNDY'] = 'Yes'

# map dictionary to sdf_units dataframe to fill in TAZ and Block Group fields
sdfParcels['TAZ']                   = sdfParcels.APN.map(dict(zip(sdf_units_taz.APN,   sdf_units_taz.TAZ_1)))
sdfParcels['BLOCK_GROUP']           = sdfParcels.APN.map(dict(zip(sdf_units_block.APN, sdf_units_block.TRPAID)))
# map IPES score to parcels
sdfParcels['IPES_SCORE']            = sdfParcels['APN'].map(dict(zip(dfIPES.APN, dfIPES.IPESScore)))
sdfParcels['IPES_SCORE_TYPE']       = sdfParcels['APN'].map(dict(zip(dfIPES.APN, dfIPES.IPESScoreType)))
# retired parcels
sdfParcels['RETIRED']               = sdfParcels['APN'].map(dict(zip(dfAllParcels.APN, dfAllParcels.RetiredFromDevelopment)))
sdfParcels['WITHIN_BONUSUNIT_BNDY'] = sdfParcels['APN'].map(dict(zip(sdf_units_bonus.APN, sdf_units_bonus.WITHIN_BONUSUNIT_BNDY)))
# define housnig zoning and density
sdfParcels['HOUSING_ZONING']          = 'NA'
sdfParcels['COMMERCIAL_ALLOWED']      = 'No'
sdfParcels['TOURIST_ALLOWED']         = 'No'

# if the zoning id is in the list of multiple family zones then set to MF
sdfParcels.loc[sdfParcels['ZONING_ID'].isin(get_sf_mf_zones(df_uses)['Zoning_ID']), 'HOUSING_ZONING'] = 'SF/MF'
# if the zoning id is in the list of single family zones and not in the multiple family zones then set to SF only
sdfParcels.loc[sdfParcels['ZONING_ID'].isin(get_sf_only_zones(df_uses)['Zoning_ID']), 'HOUSING_ZONING'] = 'SF_only'
# if the zoning id is in the list of multiple family zones and not in the single family zones then set to MF only
sdfParcels.loc[sdfParcels['ZONING_ID'].isin(get_mf_only_zones(df_uses)['Zoning_ID']), 'HOUSING_ZONING'] = 'MF_only'
# if the zoning id is in the list of commercial zones then set to Commercial
sdfParcels.loc[sdfParcels['ZONING_ID'].isin(get_commercial_zones(df_uses)['Zoning_ID']), 'COMMERCIAL_ALLOWED'] = 'Yes'
# if the zoning id is in the list of tourist zones then set to Tourist Accommodation
sdfParcels.loc[sdfParcels['ZONING_ID'].isin(get_tourist_zones(df_uses)['Zoning_ID']), 'TOURIST_ALLOWED'] = 'Yes'

# if COUNTY is in EL or PL and SF allowed then set ADU_ALLOWED to yes or if COUNTY is in WA, DG, or CC and parcel acres is greater than 1 and SF allowed then set ADU_ALLOWED to yes
sdfParcels['ADU_ALLOWED'] = 'No'
sdfParcels.loc[(sdfParcels['COUNTY'].isin(['EL','PL'])) & (~sdfParcels['HOUSING_ZONING'].isin(['MF_only', 'NA'])), 'ADU_ALLOWED'] = 'Yes'
sdfParcels.loc[(sdfParcels['COUNTY'].isin(['WA','DG','CC'])) & (sdfParcels['PARCEL_ACRES']>=1) &(~sdfParcels['HOUSING_ZONING'].isin(['MF_only', 'NA'])), 'ADU_ALLOWED'] = 'Yes'

# get density for MF and MF only zones, max residential units, and adjusted residential units
dfMF = get_mf_zones(df_uses)
sdfParcels['DENSITY']                    = sdfParcels['ZONING_ID'].map(dict(zip(dfMF.Zoning_ID, dfMF.Density)))
sdfParcels['MAX_RESIDENTIAL_UNITS']      = sdfParcels['PARCEL_ACRES'] * sdfParcels['DENSITY']
sdfParcels['MAX_UNITS']                  = sdfParcels['MAX_RESIDENTIAL_UNITS']*0.6
sdfParcels['MAX_UNITS']                  = sdfParcels['MAX_UNITS'].fillna(0).astype(int)

# set SF only zones to 1 max unit
sdfParcels.loc[sdfParcels['HOUSING_ZONING'] == 'SF_only', 'MAX_UNITS'] = 1

# set field for underbuilt evaluation
sdfParcels['POTENTIAL_UNITS'] = 0
sdfParcels['POTENTIAL_UNITS'] = sdfParcels['MAX_UNITS'] - sdfParcels['Residential_Units']
# set negative values to 0
sdfParcels.loc[sdfParcels['POTENTIAL_UNITS'] < 0, 'POTENTIAL_UNITS'] = 0

# calculate parcels with the greatest buildable potential  filter to the top 10% of parcels
# what value is in the top 10% of the potential buildable units
top_10_threshold = sdfParcels.POTENTIAL_UNITS.quantile(0.9)
# filter out rows where POTENTIAL_BUILDABLE_UNITS is NaN
sdfParcels['TOP_TEN_POTENTIAL_UNITS'] = sdfParcels.apply(lambda x: 'Yes' if x['POTENTIAL_UNITS'] >= top_10_threshold else 'No', axis=1)

# set FORECASTED_RESIDENTIAL_UNITS to 0
sdfParcels['FORECASTED_RESIDENTIAL_UNITS']     = 0
# set FORECAST_COMMERCIAN_UNITS to 0
sdfParcels['FORECASTED_COMMERCIAL_SQFT']       = 0
# set FORECAST_TOURIST_UNITS to 0
sdfParcels['FORECASTED_TOURIST_UNITS']         = 0
# set FORECAST_REASON to na
sdfParcels['FORECAST_REASON']                  = None
# FORECASTED_OCCUPANCY_RATE as a float field
sdfParcels['FORECASTED_RES_OCCUPANCY_RATE']    = 0.0

# export to pickle
sdfParcels.to_pickle(parcel_pickle_part1)
# to feature class
sdfParcels.spatial.to_featureclass(Path(gdb)/'Parcel_Base_2022')

### Functions

In [ ]:
# Get relevant census variables and calculate rates at block group level
# Get Occupancy Data - B25002_003E = Vacant, B25002_002E = Occupied , B25004_006E = Vacant Seasonal
occupancy_codes = ['B25002_003E','B25002_002E', 'B25004_006E']
df_census_occupancy = df_census_2022[df_census_2022['variable_code'].isin(occupancy_codes)]
df_census_occupancy = df_census_occupancy[['TRPAID', 'variable_code', 'value']]
# pivot to wide format so we can calculate percentages and totals
df_census_occupancy = df_census_occupancy.pivot(index='TRPAID', columns='variable_code', values='value').reset_index()
# vacant units + occupied units = total units
df_census_occupancy['total_units'] = df_census_occupancy['B25002_003E'] + df_census_occupancy['B25002_002E']
# occupancy rate = occupied units / total units
df_census_occupancy['occupancy_rate'] = df_census_occupancy['B25002_002E'] / df_census_occupancy['total_units']
# seasonal rate = seasonal units / total units
df_census_occupancy['seasonal_rate'] = df_census_occupancy['B25004_006E'] / df_census_occupancy['total_units']


# Get Household Size Data - B25010_001E = Total Households
df_census_household_size = df_census_2022[df_census_2022['variable_code'] == 'B25010_001E']
df_census_household_size = df_census_household_size[['TRPAID', 'variable_code', 'value']]
df_census_household_size = df_census_household_size.pivot(index='TRPAID', columns='variable_code', values='value').reset_index()
df_census_household_size['household_size'] = df_census_household_size['B25010_001E']
#adjust household size by a constant factor so that total residents = total population from acs
ACS_total_population = 53842
# This is from a final calculated input summary - will eventually make an explicit calculation
total_calculated_population = 53019.75
household_size_adjustment = ACS_total_population / total_calculated_population
df_census_household_size['household_size_raw'] = df_census_household_size['household_size']
df_census_household_size['household_size'] = df_census_household_size['household_size'] * household_size_adjustment


# List of Codes by the category they fall into - Census categroy to broader category
code_lookup = pd.read_csv('Lookup_Lists/income_census_codes.csv')
#Filter census so only variable codes in the code lookup are included
df_census_income = df_census_2022[df_census_2022['variable_code'].isin(code_lookup['variable_code'])]
#Create a new column that has a value from code lookup based on the variable code
df_census_income['income_category'] = df_census_income['variable_code'].map(code_lookup.set_index('variable_code')['category'])
#group by block group and income category and sum the values
df_census_income = df_census_income.groupby(['TRPAID','income_category'])['value'].sum().reset_index()
df_census_income = df_census_income.pivot(index='TRPAID', columns='income_category', values='value').reset_index()


# TRPAID is a 16 digit ID, but it is imported as a float. Convert to string and to retain leading zeros
df_census_household_size['TRPAID']= df_census_household_size['TRPAID'].astype(str).str.zfill(16)
df_census_income['TRPAID']= df_census_income['TRPAID'].astype(str).str.zfill(16)
# merge all the census data together
df_census_occupancy_all = pd.merge(df_census_occupancy, df_census_household_size, on='TRPAID', how='left')
df_census_all = pd.merge(df_census_occupancy_all, df_census_income, on='TRPAID', how='left')
# rename columns of df_census_all
column_rename = {
    'B25002_003E': 'vacant_units',
    'B25002_002E': 'occupied_units',
    'B25004_006E': 'seasonal_units',
    'High Income': 'high_income',
    'Low Income': 'low_income',
    'Medium Income': 'middle_income',
}
df_census_all.rename(columns=column_rename, inplace=True)

df_census_all.drop(columns=['B25010_001E'], inplace=True)
# calculate proportions of income categories
df_census_all['high_income_proportion'] = df_census_all['high_income'] / df_census_all['occupied_units']
df_census_all['middle_income_proportion'] = df_census_all['middle_income'] / df_census_all['occupied_units']
df_census_all['low_income_proportion'] = df_census_all['low_income'] / df_census_all['occupied_units']


# get pickle part 3
sdfParcel = pd.read_pickle(parcel_pickle_part3)

# map values from Census data to parcel data via left_on BLOCK_GROUP and right_on TRPAID
sdfParcel.PrimaryResidence_Rate   = sdfParcel.BLOCK_GROUP.map(df_census_all.set_index('TRPAID')['occupancy_rate'])
sdfParcel.SecondaryResidence_Rate = sdfParcel.BLOCK_GROUP.map(df_census_all.set_index('TRPAID')['seasonal_rate'])
sdfParcel.HighIncome_Rate         = sdfParcel.BLOCK_GROUP.map(df_census_all.set_index('TRPAID')['high_income_proportion'])
sdfParcel.MediumIncome_Rate       = sdfParcel.BLOCK_GROUP.map(df_census_all.set_index('TRPAID')['middle_income_proportion'])
sdfParcel.LowIncome_Rate          = sdfParcel.BLOCK_GROUP.map(df_census_all.set_index('TRPAID')['low_income_proportion'])
sdfParcel.PersonsPerUnit          = sdfParcel.BLOCK_GROUP.map(df_census_all.set_index('TRPAID')['household_size'])
sdfParcel['PersonsPerUnit_Raw']      = sdfParcel.BLOCK_GROUP.map(df_census_all.set_index('TRPAID')['household_size_raw'])

# seasonal rate calculation
# group by BLOCK_GROUP
# filter sdfParcel where VHR   = 'Yes'
vhrs = sdfParcel.loc[sdfParcel['VHR']=='Yes']
totalRes = sdfParcel.groupby('BLOCK_GROUP').agg({'Residential_Units':'sum', 'PrimaryResidence_Rate':'mean', 
                                                 'SecondaryResidence_Rate':'mean'}).reset_index()
totalVHR = vhrs.groupby('BLOCK_GROUP').agg({'Residential_Units':'sum'}).reset_index()
totalVHR.rename(columns={'Residential_Units':'VHR_Units'}, inplace=True)

# merge totalRes and totalVHR
totalResVHR = pd.merge(totalRes, totalVHR, on='BLOCK_GROUP', how='left')
# fill NA with 0
totalResVHR.VHR_Units = totalResVHR.VHR_Units.fillna(0)
# calculate seasonal rate
totalResVHR['non_vhr_units'] = totalResVHR['Residential_Units'] - totalResVHR['VHR_Units']
# calculate the non-adjusted number of seasonal units and then subtract the number of VHRs
totalResVHR['non_adjusted_seasonal_units'] = totalResVHR['SecondaryResidence_Rate'] * totalResVHR['Residential_Units']
totalResVHR['non_primary_residence_units'] = totalResVHR['Residential_Units']-(totalResVHR['PrimaryResidence_Rate'] * totalResVHR['Residential_Units'])
totalResVHR['adjusted_seasonal_units']     = totalResVHR['non_adjusted_seasonal_units'] - totalResVHR['VHR_Units']
# Manually adjust the seasonal units for block group 3200500170022020 because of a lag in the data
# The census reports 100% occupancy but I think it has to do with the beach club development
totalResVHR.loc[totalResVHR['BLOCK_GROUP'] == '3200500170022020', 'adjusted_seasonal_units'] = 0
# calculate the adjusted seasonal rate
totalResVHR['adjusted_seasonal_rate'] = totalResVHR['adjusted_seasonal_units'] / totalResVHR['non_primary_residence_units']

# map the adjusted seasonal rate to the parcel data
sdfParcel['SecondaryResidence_Rate'] = sdfParcel['BLOCK_GROUP'].map(totalResVHR.set_index('BLOCK_GROUP')['adjusted_seasonal_rate'])

# export to pickle part 4
sdfParcel.to_pickle(parcel_pickle_part2)
sdfParcel.info()